# 2 register device to fiware on minikube

**confirm that you have been started `iotagent-json`**

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `uoa-poc2`.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/minikube/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"
alias iso8601="python -c 'import datetime; print(datetime.datetime.now(tz=datetime.timezone.utc).isoformat(timespec=\"seconds\"))'"

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
user	tags
ros	[]
guest	[administrator]
iotagent	[]
```

## register delivery robot service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/json/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${DELIVERY_ROBOT_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/json",
      "entity_type": "${DELIVERY_ROBOT_TYPE}"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/json/manage/iot/services/ | jq .

example)
```json
{
  "count": 1,
  "services": [
    {
      "commands": [],
      "lazy": [],
      "attributes": [],
      "_id": "5d1daefe35d6f5000f7e1ed5",
      "resource": "/iot/json",
      "apikey": "delivery_robot",
      "service": "uoa_poc2",
      "subservice": "/delivery_robot",
      "__v": 0,
      "static_attributes": [],
      "internal_attributes": [],
      "entity_type": "delivery_robot"
    }
  ]
}
```

## register delivery robot device (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/json/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${DELIVERY_ROBOT_01}",
      "entity_name": "${DELIVERY_ROBOT_01}",
      "entity_type": "${DELIVERY_ROBOT_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "json",
      "attributes": [
        {
          "name": "time",
          "type": "ISO8601"
        },
        {
          "name": "mode",
          "type": "string"
        },
        {
          "name": "errors",
          "type": "array"
        },
        {
          "name": "pose",
          "type": "object"
        },
        {
          "name": "covariance",
          "type": "array"
        },
        {
          "name": "battery",
          "type": "object"
        },
        {
          "name": "current_routes",
          "type": "array"
        }
      ],
      "commands": [
        {
          "name": "send_cmd",
          "type": "object"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/json/manage/iot/devices/${DELIVERY_ROBOT_01}/ | jq .

example)
```json
{
  "device_id": "megarover_01",
  "service": "uoa_poc2",
  "service_path": "/delivery_robot",
  "entity_name": "megarover_01",
  "entity_type": "delivery_robot",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "time",
      "name": "time",
      "type": "ISO8601"
    },
    {
      "object_id": "mode",
      "name": "mode",
      "type": "string"
    },
    {
      "object_id": "errors",
      "name": "errors",
      "type": "array"
    },
    {
      "object_id": "pose",
      "name": "pose",
      "type": "object"
    },
    {
      "object_id": "covariance",
      "name": "covariance",
      "type": "array"
    },
    {
      "object_id": "battery",
      "name": "battery",
      "type": "object"
    },
    {
      "object_id": "current_routes",
      "name": "current_routes",
      "type": "array"
    }
  ],
  "lazy": [],
  "commands": [
    {
      "object_id": "send_cmd",
      "name": "send_cmd",
      "type": "object"
    }
  ],
  "static_attributes": [],
  "protocol": "json"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "covariance": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "errors": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "pose": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "time": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### register `cygnus-mongo` as a subscriber of delivery robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DELIVERY_ROBOT_ID_PREFIX}.*",
      "type": "${DELIVERY_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["time", "mode", "errors", "pose", "covariance", "battery"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["time", "mode", "errors", "pose", "covariance", "battery"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5d1db1a12997f1d49844d562
fiware-correlator: 80662fd4-9e31-11e9-9aa9-0242ac110012
date: Thu, 04 Jul 2019 07:58:25 GMT
x-envoy-upstream-service-time: 4
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DELIVERY_ROBOT_ID_PREFIX}.*",
      "type": "${DELIVERY_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["send_cmd", "send_cmd_status", "send_cmd_info"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["send_cmd", "send_cmd_status", "send_cmd_info"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5d1db3112997f1d49844d563
fiware-correlator: 5b9f0da0-9e32-11e9-8a68-0242ac110012
date: Thu, 04 Jul 2019 08:04:33 GMT
x-envoy-upstream-service-time: 4
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DELIVERY_ROBOT_ID_PREFIX}.*",
      "type": "${DELIVERY_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["current_routes"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["current_routes"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)

```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5d4a7d0e80ccc73cc8dd9ba1
fiware-correlator: 9ebfa0f0-b8e4-11e9-81af-0242ac110014
date: Wed, 07 Aug 2019 07:26:06 GMT
x-envoy-upstream-service-time: 2
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5d4a7cae84c0c749cef68823",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "megarover_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "time",
          "mode",
          "errors",
          "pose",
          "covariance",
          "battery"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-08-07T07:24:30.00Z",
      "attrs": [
        "time",
        "mode",
        "errors",
        "pose",
        "covariance",
        "battery"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-08-07T07:24:30.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5d4a7ccb5066f7a6fd0b7bfc",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "megarover_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "send_cmd",
          "send_cmd_status",
          "send_cmd_info"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-08-07T07:24:59.00Z",
      "attrs": [
        "send_cmd",
        "send_cmd_status",
        "send_cmd_info"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-08-07T07:24:59.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5d4a7d0e80ccc73cc8dd9ba1",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "megarover_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "current_routes"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-08-07T07:26:06.00Z",
      "attrs": [
        "current_routes"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-08-07T07:26:06.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## register place entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "place_01",
  "type": "place",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "name": {
    "type": "string",
    "value": "LICTiA管理室"
  },
  "position": {
    "type": "object",
    "value": {"x": 2.1, "y": 2.0, "z": 0.0}
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "place_02",
  "type": "place",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "name": {
    "type": "string",
    "value": "オープンスペース"
  },
  "position": {
    "type": "object",
    "value": {"x": 2.1, "y": 5.0, "z": 0.0}
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "place_03",
  "type": "place",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "name": {
    "type": "string",
    "value": "ホワイトボード室"
  },
  "position": {
    "type": "object",
    "value": {"x": 1.1, "y": 0.2, "z": 0.0}
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "place_04",
  "type": "place",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "name": {
    "type": "string",
    "value": "会議室1"
  },
  "position": {
    "type": "object",
    "value": {"x": 1.1, "y": 1.5, "z": 0.0}
  }
}
__EOS__

example)

```
HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/place_01?type=place
fiware-correlator: 8fb8db20-b8ae-11e9-a042-0242ac110014
date: Wed, 07 Aug 2019 00:59:08 GMT
x-envoy-upstream-service-time: 2
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/place_02?type=place
fiware-correlator: 903e3ab8-b8ae-11e9-ae14-0242ac110015
date: Wed, 07 Aug 2019 00:59:09 GMT
x-envoy-upstream-service-time: 3
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/place_03?type=place
fiware-correlator: 90bf1656-b8ae-11e9-b8e3-0242ac110013
date: Wed, 07 Aug 2019 00:59:10 GMT
x-envoy-upstream-service-time: 2
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/place_04?type=place
fiware-correlator: 913e11f4-b8ae-11e9-a042-0242ac110014
date: Wed, 07 Aug 2019 00:59:10 GMT
x-envoy-upstream-service-time: 2
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/?type=place | jq .

example)
```json
[
  {
    "id": "place_01",
    "type": "place",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T07:32:25.00Z",
      "metadata": {}
    },
    "name": {
      "type": "string",
      "value": "LICTiA管理室",
      "metadata": {}
    },
    "position": {
      "type": "object",
      "value": {
        "x": 2.1,
        "y": 2,
        "z": 0
      },
      "metadata": {}
    }
  },
  {
    "id": "place_02",
    "type": "place",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T07:32:26.00Z",
      "metadata": {}
    },
    "name": {
      "type": "string",
      "value": "オープンスペース",
      "metadata": {}
    },
    "position": {
      "type": "object",
      "value": {
        "x": 2.1,
        "y": 5,
        "z": 0
      },
      "metadata": {}
    }
  },
  {
    "id": "place_03",
    "type": "place",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T07:32:27.00Z",
      "metadata": {}
    },
    "name": {
      "type": "string",
      "value": "ホワイトボード室",
      "metadata": {}
    },
    "position": {
      "type": "object",
      "value": {
        "x": 1.1,
        "y": 0.2,
        "z": 0
      },
      "metadata": {}
    }
  },
  {
    "id": "place_04",
    "type": "place",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T07:32:28.00Z",
      "metadata": {}
    },
    "name": {
      "type": "string",
      "value": "会議室1",
      "metadata": {}
    },
    "position": {
      "type": "object",
      "value": {
        "x": 1.1,
        "y": 1.5,
        "z": 0
      },
      "metadata": {}
    }
  }
]
```

## register route_plan entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "route_plan_01",
  "type": "route_plan",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "destination": {
    "type": "string",
    "value": "place_01"
  },
  "via": {
    "type": "string",
    "value": "place_03"
  },
  "routes": {
    "type": "array",
    "value": [
      [{"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 2.1, "y": 2.0, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 0.0, "y": 0.0, "z": 0.0}]
    ]
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "route_plan_02",
  "type": "route_plan",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "destination": {
    "type": "string",
    "value": "place_01"
  },
  "via": {
    "type": "string",
    "value": "place_04"
  },
  "routes": {
    "type": "array",
    "value": [
      [{"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 1.5, "z": 0.0}],
      [{"x": 2.1, "y": 2.0, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 0.0, "y": 0.0, "z": 0.0}]
    ]
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "route_plan_03",
  "type": "route_plan",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "destination": {
    "type": "string",
    "value": "place_01"
  },
  "via": {
    "type": "string",
    "value": "place_03|place_04"
  },
  "routes": {
    "type": "array",
    "value": [
      [{"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}],
      [{"x": 2.1, "y": 2.0, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 0.0, "y": 0.0, "z": 0.0}]
    ]
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "route_plan_04",
  "type": "route_plan",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "destination": {
    "type": "string",
    "value": "place_02"
  },
  "via": {
    "type": "string",
    "value": "place_03"
  },
  "routes": {
    "type": "array",
    "value": [
      [{"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 2.1, "y": 5.0, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 0.0, "y": 0.0, "z": 0.0}]
    ]
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "route_plan_05",
  "type": "route_plan",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "destination": {
    "type": "string",
    "value": "place_02"
  },
  "via": {
    "type": "string",
    "value": "place_04"
  },
  "routes": {
    "type": "array",
    "value": [
      [{"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 1.5, "z": 0.0}],
      [{"x": 2.1, "y": 5.0, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 0.0, "y": 0.0, "z": 0.0}]
    ]
  }
}
__EOS__
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": "route_plan_06",
  "type": "route_plan",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "destination": {
    "type": "string",
    "value": "place_02"
  },
  "via": {
    "type": "string",
    "value": "place_03|place_04"
  },
  "routes": {
    "type": "array",
    "value": [
      [{"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}],
      [{"x": 2.1, "y": 5.0, "z": 0.0}],
      [{"x": 1.1, "y": 1.5, "z": 0.0}, {"x": 1.1, "y": 0.2, "z": 0.0}, {"x": 0.1, "y": 0.2, "z": 0.0}, {"x": 0.0, "y": 0.0, "z": 0.0}]
    ]
  }
}
__EOS__

example)

```
HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_01?type=route_plan
fiware-correlator: ca48699a-b8ae-11e9-b8e3-0242ac110013
date: Wed, 07 Aug 2019 01:00:46 GMT
x-envoy-upstream-service-time: 3
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_02?type=route_plan
fiware-correlator: cb2ae216-b8ae-11e9-8a3d-0242ac110014
date: Wed, 07 Aug 2019 01:00:48 GMT
x-envoy-upstream-service-time: 2
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_03?type=route_plan
fiware-correlator: cc1445d2-b8ae-11e9-81af-0242ac110014
date: Wed, 07 Aug 2019 01:00:49 GMT
x-envoy-upstream-service-time: 2
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_04?type=route_plan
fiware-correlator: ccf5622e-b8ae-11e9-a896-0242ac110014
date: Wed, 07 Aug 2019 01:00:51 GMT
x-envoy-upstream-service-time: 2
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_05?type=route_plan
fiware-correlator: cdd8fa34-b8ae-11e9-8a3d-0242ac110014
date: Wed, 07 Aug 2019 01:00:52 GMT
x-envoy-upstream-service-time: 2
server: envoy

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_06?type=route_plan
fiware-correlator: cec5a744-b8ae-11e9-8a3d-0242ac110014
date: Wed, 07 Aug 2019 01:00:54 GMT
x-envoy-upstream-service-time: 2
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/?type=route_plan | jq .

example)
```json
[
  {
    "id": "route_plan_01",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T01:00:46.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_01",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        [
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 2.1,
            "y": 2,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0,
            "y": 0,
            "z": 0
          }
        ]
      ],
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_03",
      "metadata": {}
    }
  },
  {
    "id": "route_plan_02",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T01:00:48.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_01",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        [
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          }
        ],
        [
          {
            "x": 2.1,
            "y": 2,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0,
            "y": 0,
            "z": 0
          }
        ]
      ],
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_04",
      "metadata": {}
    }
  },
  {
    "id": "route_plan_03",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T01:00:49.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_01",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        [
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          }
        ],
        [
          {
            "x": 2.1,
            "y": 2,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0,
            "y": 0,
            "z": 0
          }
        ]
      ],
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_03|place_04",
      "metadata": {}
    }
  },
  {
    "id": "route_plan_04",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T01:00:51.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_02",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        [
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 2.1,
            "y": 5,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0,
            "y": 0,
            "z": 0
          }
        ]
      ],
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_03",
      "metadata": {}
    }
  },
  {
    "id": "route_plan_05",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T01:00:52.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_02",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        [
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          }
        ],
        [
          {
            "x": 2.1,
            "y": 5,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0,
            "y": 0,
            "z": 0
          }
        ]
      ],
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_04",
      "metadata": {}
    }
  },
  {
    "id": "route_plan_06",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-08-07T01:00:54.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_02",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        [
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          }
        ],
        [
          {
            "x": 2.1,
            "y": 5,
            "z": 0
          }
        ],
        [
          {
            "x": 1.1,
            "y": 1.5,
            "z": 0
          },
          {
            "x": 1.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0.1,
            "y": 0.2,
            "z": 0
          },
          {
            "x": 0,
            "y": 0,
            "z": 0
          }
        ]
      ],
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_03|place_04",
      "metadata": {}
    }
  }
]
```

## test subscribing the attributes of delivery robot (megarover_01)

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### publish a MQTT message to emulate delivery robot action

In [ ]:
m=$(cat << __EOS__ 
{
  "time": "$(now)",
  "mode": "navi",
  "errors": [],
  "pose": {
    "point": {
      "x": 3.402,
      "y": 1.015,
      "z": -0.002
    },
    "angle": {
      "roll": -0.001,
      "pitch": 0.012,
      "yaw": 1.581
    }
  },
  "covariance": [
    0.1, 0.0, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.1, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0, 0.0,
    0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0,
    0.0, 0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0,
    0.0, 0.0, 0.0, 0.0, 0.0, 0.05
  ],
  "battery": {
    "voltage": 11.495
  }
}
__EOS__
)
mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/attrs -m "$m"

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/63ZwU16FdHZ67aPkHS received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/attrs', ... (588 bytes))
{
  "time": "2019-08-07T16:37:16.323513+09:00",
  "mode": "navi",
  "errors": [],
  "pose": {
    "point": {
      "x": 3.402,
      "y": 1.015,
      "z": -0.002
    },
    "angle": {
      "roll": -0.001,
      "pitch": 0.012,
      "yaw": 1.581
    }
  },
  "covariance": [
    0.1, 0.0, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.1, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0, 0.0,
    0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0,
    0.0, 0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0,
    0.0, 0.0, 0.0, 0.0, 0.0, 0.05
  ],
  "battery": {
    "voltage": 11.495
  }
}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-07T07:37:16.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495"
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-07T16:37:16.323513+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
c=$(echo sth_${DELIVERY_ROBOT_SERVICEPATH}xffff${DELIVERY_ROBOT_01}xffff${DELIVERY_ROBOT_TYPE} | perl -pe 's/\//x002f/g; s/\$/x0024/g; s/=/xffff/g;')
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"${c}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/sth_uoa_poc2?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("00b0e8d6-7cc9-4ebe-b1de-02002a0401ff") }
MongoDB server version: 4.1.13
{ "_id" : ObjectId("5d4a7fae1283940016d8af46"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "time", "attrType" : "ISO8601", "attrValue" : "2019-08-07T16:37:16.323513+09:00" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af47"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "mode", "attrType" : "string", "attrValue" : "navi" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af48"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "pose", "attrType" : "object", "attrValue" : "{\"point\":{\"x\":\"3.402\",\"y\":\"1.015\",\"z\":\"-0.002\"},\"angle\":{\"roll\":\"-0.001\",\"pitch\":\"0.012\",\"yaw\":\"1.581\"}}" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af49"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "covariance", "attrType" : "array", "attrValue" : "[\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.05\"]" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af4a"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "battery", "attrType" : "object", "attrValue" : "{\"voltage\":\"11.495\"}" }
{ "_id" : ObjectId("5d4a7cd00c76920016cabc4f"), "recvTime" : ISODate("2019-08-07T07:24:59.402Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
```

### confirm sth-comet

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
for a in "time" "mode" "pose" "covariance" "battery"; do
  curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/comet/STH/v1/contextEntities/type/${DELIVERY_ROBOT_TYPE}/id/${DELIVERY_ROBOT_01}/attributes/${a}?lastN=100 | jq .
done

example)
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "time",
            "values": [
              {
                "recvTime": "2019-08-07T07:37:16.489Z",
                "attrType": "ISO8601",
                "attrValue": "2019-08-07T16:37:16.323513+09:00"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "mode",
            "values": [
              {
                "recvTime": "2019-08-07T07:37:16.489Z",
                "attrType": "string",
                "attrValue": "navi"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "pose",
            "values": [
              {
                "recvTime": "2019-08-07T07:37:16.489Z",
                "attrType": "object",
                "attrValue": "{\"point\":{\"x\":\"3.402\",\"y\":\"1.015\",\"z\":\"-0.002\"},\"angle\":{\"roll\":\"-0.001\",\"pitch\":\"0.012\",\"yaw\":\"1.581\"}}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "covariance",
            "values": [
              {
                "recvTime": "2019-08-07T07:37:16.489Z",
                "attrType": "array",
                "attrValue": "[\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.05\"]"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "battery",
            "values": [
              {
                "recvTime": "2019-08-07T07:37:16.489Z",
                "attrType": "object",
                "attrValue": "{\"voltage\":\"11.495\"}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```

## test publishing the `send_cmd` command of delivery robot (megarover_01)

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### send a NGSI message to emulate sending command

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/attrs?type=${DELIVERY_ROBOT_TYPE} -X PATCH -d @-<<__EOS__
{
  "send_cmd": {
    "value": {
      "time": "$(now)",
      "cmd": "navi",
      "waypoints": [
        {
          "x": 0.503,
          "y": 0.0,
          "z": 0.0
        },
        {
          "x": 3.411,
          "y": 0.0,
          "z": 0.0
        },
        {
          "x": 3.411,
          "y": 2.81,
          "z": 0.0
        }
      ]
    }
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: bbbc2f00-9eab-11e9-9611-0242ac110012
date: Thu, 04 Jul 2019 22:33:23 GMT
x-envoy-upstream-service-time: 42
server: envoy
```

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/63ZwU16FdHZ67aPkHS received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmd', ... (158 bytes))
{"send_cmd":{"time":"2019-08-07T16:39:34.065648+09:00","cmd":"navi","waypoints":[{"x":0.503,"y":0,"z":0},{"x":3.411,"y":0,"z":0},{"x":3.411,"y":2.81,"z":0}]}}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` is `PENDING`

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-07T07:39:34.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495"
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:39:34.172Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-07T16:37:16.323513+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### publish a MQTT message to emulate receiving command result

In [ ]:
m=$(cat << __EOS__ 
{
  "send_cmd": {
    "time": "$(now)",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "x": 0.503,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 2.81,
        "z": 0.0
      }
    ],
    "result": "ack",
    "errors": []
  }
}
__EOS__
)
mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/cmdexe -m "${m}"

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/63ZwU16FdHZ67aPkHS received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmdexe', ... (449 bytes))
{
  "send_cmd": {
    "time": "2019-08-07T16:43:16.846477+09:00",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "x": 0.503,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 2.81,
        "z": 0.0
      }
    ],
    "result": "ack",
    "errors": []
  }
}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` turns to `OK` and `send_cmd_info` has the result object

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-07T07:43:16.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495"
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": {
      "time": "2019-08-07T16:43:16.846477+09:00",
      "received_time": "2019-07-05T07:28:26.1562279306+09:00",
      "received_cmd": "navi",
      "received_waypoints": [
        {
          "x": "0.503",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "2.81",
          "z": "0"
        }
      ],
      "result": "ack",
      "errors": ""
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:43:16.999Z"
      }
    }
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:43:16.999Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-07T16:37:16.323513+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-07T07:37:16.489Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
c=$(echo sth_${DELIVERY_ROBOT_SERVICEPATH}xffff${DELIVERY_ROBOT_01}xffff${DELIVERY_ROBOT_TYPE} | perl -pe 's/\//x002f/g; s/\$/x0024/g; s/=/xffff/g;')
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"${c}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/sth_uoa_poc2?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("fae13fff-4c42-4e6d-9d91-02ba9c1b7e5d") }
MongoDB server version: 4.1.13
{ "_id" : ObjectId("5d4a81150c76920016cabc50"), "recvTime" : ISODate("2019-08-07T07:43:16.999Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "OK" }
{ "_id" : ObjectId("5d4a81150c76920016cabc51"), "recvTime" : ISODate("2019-08-07T07:43:16.999Z"), "attrName" : "send_cmd_info", "attrType" : "commandResult", "attrValue" : "{\"time\":\"2019-08-07T16:43:16.846477+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"x\":\"0.503\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"}],\"result\":\"ack\",\"errors\":\"\"}" }
{ "_id" : ObjectId("5d4a80361283940016d8af4b"), "recvTime" : ISODate("2019-08-07T07:39:34.172Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "PENDING" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af46"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "time", "attrType" : "ISO8601", "attrValue" : "2019-08-07T16:37:16.323513+09:00" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af47"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "mode", "attrType" : "string", "attrValue" : "navi" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af48"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "pose", "attrType" : "object", "attrValue" : "{\"point\":{\"x\":\"3.402\",\"y\":\"1.015\",\"z\":\"-0.002\"},\"angle\":{\"roll\":\"-0.001\",\"pitch\":\"0.012\",\"yaw\":\"1.581\"}}" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af49"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "covariance", "attrType" : "array", "attrValue" : "[\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.05\"]" }
{ "_id" : ObjectId("5d4a7fae1283940016d8af4a"), "recvTime" : ISODate("2019-08-07T07:37:16.489Z"), "attrName" : "battery", "attrType" : "object", "attrValue" : "{\"voltage\":\"11.495\"}" }
{ "_id" : ObjectId("5d4a7cd00c76920016cabc4f"), "recvTime" : ISODate("2019-08-07T07:24:59.402Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
```

### confirm sth-comet

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
for a in "send_cmd" "send_cmd_status" "send_cmd_info"; do
  curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/comet/STH/v1/contextEntities/type/${DELIVERY_ROBOT_TYPE}/id/${DELIVERY_ROBOT_01}/attributes/${a}?lastN=100 | jq .
done

example)
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd",
            "values": []
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd_status",
            "values": [
              {
                "recvTime": "2019-08-07T07:24:59.402Z",
                "attrType": "commandStatus",
                "attrValue": "UNKNOWN"
              },
              {
                "recvTime": "2019-08-07T07:39:34.172Z",
                "attrType": "commandStatus",
                "attrValue": "PENDING"
              },
              {
                "recvTime": "2019-08-07T07:43:16.999Z",
                "attrType": "commandStatus",
                "attrValue": "OK"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd_info",
            "values": [
              {
                "recvTime": "2019-08-07T07:43:16.999Z",
                "attrType": "commandResult",
                "attrValue": "{\"time\":\"2019-08-07T16:43:16.846477+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"x\":\"0.503\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"0\",\"z\":\"0\"},{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"}],\"result\":\"ack\",\"errors\":\"\"}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```